In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
FTRAIN = 'data/train.json'
FTEST = 'data/test.json'

In [3]:
df_train = pd.read_json(FTRAIN)

In [4]:
df_test = pd.read_json(FTEST)

In [5]:
def get_longest_features(df, num_results = 10):
    return feature_counts.loc[feature_counts['name'].apply(len).sort_values(ascending = False).head(num_results).index, :]

In [6]:
def extract_features(feature_parser = lambda x: [x.lower()]):
    features = {}
    for df in [df_train, df_test]:
        for flist in df['features']:
            for ft in flist:
                for ft_parsed in feature_parser(ft):
                    features[ft_parsed] = features.get(ft_parsed, 0) + 1
    return pd.DataFrame([[k, v] for k, v in features.iteritems()], columns = ['name', 'counts'])

In [7]:
def subparser(x):
    x = x.lower().replace('-', ' ').strip()
    if x[0] == '{':
        return [y.replace('"', '').strip() for y in re.findall(r'(?<=\d\s=\s)([^;]+);', x)]
    x = x.split(u'\u2022')
    return [z for y in x for z in re.split(r'[\.\s!]!*\s+|\s+-\s+|\s*\*\s*', y)]

def parser(x):
    return [z for z in [y.strip() for y in subparser(x)] if len(z) > 0]

In [8]:
feature_counts = extract_features(parser)

In [9]:
len(feature_counts)

2634

In [10]:
for name in get_longest_features(feature_counts, 50)['name']:
    print name

bike room driveway garage garden fitness facility(fee) spa services pool sauna steam room laundry on each flr valet playroom lounge billiards room rooftop deck common storage wifi access
east and west resident lounges reading room outdoor lounges with river views party room chefs kitchen movie screening room tot spot playroom ping pong
bike room driveway garage water filtration fitness room pool sauna laundry on each flr housekeeping valet playroom lounge rooftop deck common storage
bike room garage garden fitness facility(fee) spa services valet playroom lounge party room rooftop deck private storage wifi access business center
residents only fitness center and aerobic room professionally outfitted with a full complement of strength and cardio training equipment
and stainless steel appliances with a dishwasher well maintained hardwood floors add to the warmth and modern feel of the space.
bike room garage fitness facility laundry room housekeeping valet lounge party room rooftop deck 